In [1]:
import os, json
from unstructured.partition.auto import partition


In [2]:

pdf_path = "./质量协议-朗恩斯 2023版模板-20230213法务.pdf"

elements = partition(filename=pdf_path)





/root/autodl-tmp/envs/doc-parse/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

print("\n\n".join([str(el) for el in elements]))


for index, el in enumerate(elements):
    print(type(el))
    # if index > 2 and index < 50:
    #     print(el)
    


江苏中科朗恩斯车辆科技有限公司 LANCE VEHICLE TECHONOLOGY CO.,LTD

Supplier Quality Agreement

编号：

甲方(需方)：江苏中科朗恩斯车辆科技有限公司

Party A：

LANCE VEHICLE TECHONOLOGY CO.,LTD.

乙方(供方)：

Party B：

第 1 页 共 19 页

江苏中科朗恩斯车辆科技有限公司 LANCE VEHICLE TECHONOLOGY CO.,LTD

甲方与乙方本着长期合作、平等互利、风险共担和共同发展的原则， 为确保所有供货产

品在开发、认可和批量生产供货全过程中质量得到保证， 甲乙双方达成以下协议， 共同信守

执行。

乙方对所提供产品的质量负责，需要满足但不局限于以下的要求：

2.1 乙方提供给甲方的产品、过程和服务应符合国家相关标准、客户标准、行业标准及

法律法规(符合收货国、发运国和甲方确定的目的国现行适用的法律法规要求)；

2.2 对有安全要求的产品应提供有效的安全认证证书，并按相关规定在产品上注明安全

认证标志；

2.3 甲方书面确认的产品技术要求、图纸、产品标准、 封样等；

2.4 甲方关于物料禁用物质的管理要求；

2.5 甲方的供应商管理文件；

2.6 图纸更改，技术质量改进，应按甲方的《设计变更控制程序》和《工程变更控制程

序》执行；

2.7 满足国家安全环保相关技术指标，满足国家政府对汽车零部件的有关法律法规；

2.8 符合《中华人民共和国产品质量法》。

针对产品的特殊要求， 甲方可以和乙方签署质量补充协议， 作为此质量协议的附件， 和

本质量协议具有同等效力。

3.1 乙方应有健全的质量体系，通过

IATF16949

质量管理体系认证;

3.2 要求乙方通过 IATF16949:2016 的认证，最低限度要通过 ISO9001:2015 的认证。未

通过 IATF16949:2016 认证的， 乙方需提交认证计划给甲方认可,并在供货月起一年内取得质

量体系证书,具体参考 “IATF16949:2016 8.4.2.3 供应商质量管理体系开发”的要求执行。

第 2 页 共 19 页

江苏中科朗恩斯车辆科技有限公司 LANCE VEHICLE TECHONOLOGY CO.,LTD

In [4]:



import pymupdf # imports the pymupdf library


all_texts = []
doc = pymupdf.open("./质量协议-朗恩斯 2023版模板-20230213法务.pdf") # open a document
for page in doc: # iterate the document pages
    text = page.get_text() # get plain text encoded as UTF-8
    print(text, "--")
    all_texts.append(text)




ModuleNotFoundError: No module named 'pymupdf'

In [ ]:

doc_text = "".join(all_texts[:30])
doc_text



In [ ]:

# https://github.com/opendatalab/MinerU/blob/master/demo/magic_pdf_parse_main.py#L50

import os
import json
import copy

from loguru import logger

from magic_pdf.pipe.UNIPipe import UNIPipe
from magic_pdf.pipe.OCRPipe import OCRPipe
from magic_pdf.pipe.TXTPipe import TXTPipe
from magic_pdf.rw.DiskReaderWriter import DiskReaderWriter
import magic_pdf.model as model_config

model_config.__use_inside_model__ = True

# todo: 设备类型选择 （？）

def json_md_dump(
        pipe,
        md_writer,
        pdf_name,
        content_list,
        md_content,
):
    # 写入模型结果到 model.json
    orig_model_list = copy.deepcopy(pipe.model_list)
    md_writer.write(
        content=json.dumps(orig_model_list, ensure_ascii=False, indent=4),
        path=f"{pdf_name}_model.json"
    )

    # 写入中间结果到 middle.json
    md_writer.write(
        content=json.dumps(pipe.pdf_mid_data, ensure_ascii=False, indent=4),
        path=f"{pdf_name}_middle.json"
    )

    # text文本结果写入到 conent_list.json
    md_writer.write(
        content=json.dumps(content_list, ensure_ascii=False, indent=4),
        path=f"{pdf_name}_content_list.json"
    )

    # 写入结果到 .md 文件中
    md_writer.write(
        content=md_content,
        path=f"{pdf_name}.md"
    )


def pdf_parse_main(
        pdf_path: str,
        parse_method: str = 'auto',
        model_json_path: str = None,
        is_json_md_dump: bool = True,
        output_dir: str = None
):
    """
    执行从 pdf 转换到 json、md 的过程，输出 md 和 json 文件到 pdf 文件所在的目录

    :param pdf_path: .pdf 文件的路径，可以是相对路径，也可以是绝对路径
    :param parse_method: 解析方法， 共 auto、ocr、txt 三种，默认 auto，如果效果不好，可以尝试 ocr
    :param model_json_path: 已经存在的模型数据文件，如果为空则使用内置模型，pdf 和 model_json 务必对应
    :param is_json_md_dump: 是否将解析后的数据写入到 .json 和 .md 文件中，默认 True，会将不同阶段的数据写入到不同的 .json 文件中（共3个.json文件），md内容会保存到 .md 文件中
    :param output_dir: 输出结果的目录地址，会生成一个以 pdf 文件名命名的文件夹并保存所有结果
    """
    try:
        pdf_name = os.path.basename(pdf_path).split(".")[0]
        pdf_path_parent = os.path.dirname(pdf_path)

        if output_dir:
            output_path = os.path.join(output_dir, pdf_name)
        else:
            output_path = os.path.join(pdf_path_parent, pdf_name)

        output_image_path = os.path.join(output_path, 'images')

        # 获取图片的父路径，为的是以相对路径保存到 .md 和 conent_list.json 文件中
        image_path_parent = os.path.basename(output_image_path)

        pdf_bytes = open(pdf_path, "rb").read()  # 读取 pdf 文件的二进制数据

        if model_json_path:
            # 读取已经被模型解析后的pdf文件的 json 原始数据，list 类型
            model_json = json.loads(open(model_json_path, "r", encoding="utf-8").read())
        else:
            model_json = []

        # 执行解析步骤
        # image_writer = DiskReaderWriter(output_image_path)
        image_writer, md_writer = DiskReaderWriter(output_image_path), DiskReaderWriter(output_path)

        # 选择解析方式
        # jso_useful_key = {"_pdf_type": "", "model_list": model_json}
        # pipe = UNIPipe(pdf_bytes, jso_useful_key, image_writer)
        if parse_method == "auto":
            jso_useful_key = {"_pdf_type": "", "model_list": model_json}
            pipe = UNIPipe(pdf_bytes, jso_useful_key, image_writer)
        elif parse_method == "txt":
            pipe = TXTPipe(pdf_bytes, model_json, image_writer)
        elif parse_method == "ocr":
            pipe = OCRPipe(pdf_bytes, model_json, image_writer)
        else:
            logger.error("unknown parse method, only auto, ocr, txt allowed")
            exit(1)

        # 执行分类
        pipe.pipe_classify()

        # 如果没有传入模型数据，则使用内置模型解析
        if not model_json:
            if model_config.__use_inside_model__:
                pipe.pipe_analyze()  # 解析
            else:
                logger.error("need model list input")
                exit(1)

        # 执行解析
        pipe.pipe_parse()

        # 保存 text 和 md 格式的结果
        content_list = pipe.pipe_mk_uni_format(image_path_parent, drop_mode="none")
        md_content = pipe.pipe_mk_markdown(image_path_parent, drop_mode="none")


        if is_json_md_dump:
            json_md_dump(pipe, md_writer, pdf_name, content_list, md_content)


    except Exception as e:
        logger.exception(e)


# # 测试
# if __name__ == '__main__':
#     pdf_path = r"C:\Users\XYTK2\Desktop\2024-2016-gb-cd-300.pdf"
#     pdf_parse_main(pdf_path)

pdf_parse_main(pdf_path="./质量协议-朗恩斯 2023版模板-20230213法务.pdf")
